# DE-Sim performance test

This notebook conducts a performance of DE-Sim simulating a model of a cyclic messaging network over a range of network sizes.

In [1]:
!pip install de_sim

ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.7/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent

The cyclic messaging network model consists of a ring of simulation objects. Each simulation object executes an event at every time unit and schedules an event for the next object in the ring 1 time unit in the future. 

In [13]:
# DE-Sim performance test
import time

import de_sim


class InitMsg(de_sim.EventMessage):
    'An InitMsg message'

def obj_name(i):
    return f'sim_obj_{i}'


class CyclicalMessagesSimulationObject(de_sim.SimulationObject):
    """ Send events around a cycle of objects """

    def __init__(self, name, obj_num, cycle_size):
        super().__init__(name)
        self.obj_num = obj_num
        self.cycle_size = cycle_size

    def next_obj(self):
        next = (self.obj_num + 1) % self.cycle_size
        return self.simulator.simulation_objects[obj_name(next)]

    def send_next_event(self):
        # send event to next CyclicalMessagesSimulationObject
        self.send_event(1, self.next_obj(), InitMsg())

    def init_before_run(self):
        self.send_next_event()

    def handle_event(self, event):
        self.send_next_event()

    event_handlers = [(InitMsg, 'handle_event')]

    # register the message types sent
    messages_sent = (InitMsg, )


def make_cyclical_messaging_network_sim(simulator, num_objs):
    # make a simulation with cyclical messaging network
    sim_objects = [CyclicalMessagesSimulationObject(obj_name(i), i, num_objs)
                   for i in range(num_objs)]
    simulator.add_objects(sim_objects)

def prep_simulation(simulator, num_sim_objs):
    simulator.reset()
    make_cyclical_messaging_network_sim(simulator, num_sim_objs)
    simulator.initialize()

def test_performance():
    simulator = de_sim.Simulator()
    end_sim_time = 100
    num_sim_objs = 4
    max_num_profile_objects = 300
    max_num_sim_objs = 5000
    print()
    print(f"Performance test of cyclical messaging network: "
          f"end simulation time: {end_sim_time}")
    unprofiled_perf = ["\n#sim. obs\t# events\trun-time (s)\tevents/s".expandtabs(14)]

    while num_sim_objs < max_num_sim_objs:

        # measure execution time
        prep_simulation(simulator, num_sim_objs)
        start_time = time.process_time()
        num_events = simulator.simulate(end_sim_time).num_events
        run_time = time.process_time() - start_time
        unprofiled_perf.append(f"{num_sim_objs:>9}\t{num_events:>7}\t"
                               f"{run_time:11.3f}\t{num_events / run_time:8.0f}".expandtabs(14))
        num_sim_objs *= 4

    print("\n".join(unprofiled_perf))

test_performance()


Performance test of cyclical messaging network: end simulation time: 100

#sim. obs     # events      run-time (s)  events/s
        4         400             0.018      21657
       16        1600             0.076      21088
       64        6400             0.314      20354
      256       25600             1.231      20792
     1024      102400             5.105      20060
     4096      409600            24.414      16777


Each simulation run executes for 100 time units.
The number of simulation objects in the ring is given by `#sim. obs`, and the number of simulation events executed by `# events`.